# Introducción

En este trabajo, exploraremos múltiples técnicas de procesamiento de imágenes para mejorar la visibilidad de indicadores patológicos en plantas.

# Exploración de datos

Primero, veamos la estructura de los datos y una descripción estadística de éstos, con el fin de conocer un poco más de las imágenes de las plantas que presentan o no enfermedades y así determinar cómo mejorar su visibilidad.

## Importación de las librerías necesarias

Importamos las librerías necesarias para leer los datos, explorarlos y procesarlos.

In [ ]:
import os

import pandas as pd
import cv2
import numpy as np


import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

np.random.seed(2022)

Definimos algunos parámetros necesarios para acceder a los datos y a para explorar las imágenes.

In [ ]:
TRAIN_PATH = "../input/plant-pathology-2020-fgvc7/train.csv"
TEST_PATH = "../input/plant-pathology-2020-fgvc7/test.csv"
IMAGE_PATH = "../input/plant-pathology-2020-fgvc7/images/"
SAMPLE_SIZE = 100

Leemos los datos de entrenamiento para acceder a las imágenes mediante el identificador de la imagen *image_id*. Vemos también un poco la estructura de los datos, con las siguientes columnas:
- ***healthy***: Indica si la planta en la imagen está sana.
- ***multiple_diseases***: Indica si la planta presenta múltiples enfermedades.
- ***rust***: Indica si la planta presenta el síntoma de "oxidación" en sus hojas.
- ***scab***: Indica si la planta tiene el síntoma de "costra" en sus hojas.

In [ ]:
train_data = pd.read_csv(TRAIN_PATH)

train_data.head()

## Carga de imágenes en memoria

Carguemos las imágenes de nuestro dataset para dar un vistazo general a las plantas presentes en éste y así plantear estrategias para procesar tales imágenes con las metodologías vistas en clase.

In [ ]:
def load_image(image_id):
    file_path = image_id + ".jpg"
    image = cv2.imread(IMAGE_PATH + file_path)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

healthy_sample = train_data[:][:SAMPLE_SIZE].query("healthy == 1")
scab_sample = train_data[:][:SAMPLE_SIZE].query("scab == 1")
rust_sample = train_data[:][:SAMPLE_SIZE].query("rust == 1")

train_images = train_data["image_id"][:SAMPLE_SIZE].apply(load_image)
healthy_images = train_images.loc[list(healthy_sample.index)]
scab_images = train_images.loc[list(scab_sample.index)]
rust_images = train_images.loc[list(rust_sample.index)]

## Visualización de las imágenes de hojas

Una vez cargados los datos, veamos algunas de las imágenes que están presentes en el dataset.

In [ ]:
random_sample = np.random.choice(train_images, 6, replace=True)

fig,ax = plt.subplots(nrows=2, ncols=3, figsize=(30, 16))
for col in range(3):
    for row in range(2):
        ax[row,col].imshow(random_sample[row*3+col])
        ax[row,col].set_xticks([])
        ax[row,col].set_yticks([])

## Distribución de canales

Una hoja sana es, usualmente, **verde**. Por otro lado, las manchas y costras que se presentan en una planta enferma suelen ser de **color marrón**, el cual en el modelo RGB es un color compuesto a partir del **verde y el rojo**, con menor apariencia del azul. Para ayudarnos un poco, usemos la librería Plotly para ver interactivamente los valores RGB en diferentes puntos de la imagen.

In [ ]:
def create_histogram(img, colors=['red', 'green', 'blue']):
    fig = go.Figure()
    for channel, color in enumerate(colors):
        fig.add_trace(go.Histogram(x=img[..., channel].ravel(), opacity=0.5,
                                   marker_color=color, name='%s channel' %color))
    return fig

### Planta sana

Primero, veamos la imagen de la hoja de una planta sana.

In [ ]:
img = healthy_images.iloc[0]
# fig = px.imshow(img)
# fig.show()

fig = make_subplots(1, 2)
# We use go.Image because subplots require traces, whereas px functions return a figure
fig.add_trace(go.Image(z=img), 1, 1)
for channel, color in enumerate(['red', 'green', 'blue']):
    fig.add_trace(go.Histogram(x=img[..., channel].ravel(), opacity=0.5,
                               marker_color=color, name='%s channel' %color), 1, 2)

fig.show()

En esta planta sana, el color **verde** es el predominante en intensidad.

### Planta con _rust_

El nombre _rust_ viene del color visto en las manchas de las hojas, el cual es un marrón _oxidado_ o amarillo. Podemos suponer que al ser de color marrón, veremos mucho más intensidad de rojos.

In [ ]:
img = rust_images.iloc[0]
# fig = px.imshow(img)
# fig.show()

fig = make_subplots(1, 2)
# We use go.Image because subplots require traces, whereas px functions return a figure
fig.add_trace(go.Image(z=img), 1, 1)
for channel, color in enumerate(['red', 'green', 'blue']):
    fig.add_trace(go.Histogram(x=img[..., channel].ravel(), opacity=0.5,
                               marker_color=color, name='%s channel' %color), 1, 2)

fig.show()

Comparado con la planta sana, vemos mucho más frecuente el **rojo** con mayor intensidad (~120 en este caso). También es posible observar que los bordes amarillos reducen el valor del **azul**, mientras que las manchas marrones más desarrolladas reducen la intensidad del **verde**.

### Planta con _scab_

Estas enfermedades son producidas por hongos o bacterias y producen _costras_ o _scabs_. Estas costras suelen ser de color marrón o casi negras, pero no suelen ser amarillas.

In [ ]:
img = scab_images.iloc[0]
# fig = px.imshow(img)
# fig.show()

fig = make_subplots(1, 2)
# We use go.Image because subplots require traces, whereas px functions return a figure
fig.add_trace(go.Image(z=img), 1, 1)
for channel, color in enumerate(['red', 'green', 'blue']):
    fig.add_trace(go.Histogram(x=img[..., channel].ravel(), opacity=0.5,
                               marker_color=color, name='%s channel' %color), 1, 2)
    
fig.show()

En el marrón encontrado en el _scab_, podemos ver que el **azul** y el **rojo** suelen subir en estas costras. Con esto, podemos ir generando hipótesis acerca de los patrones que se pueden encontrar en este tipo de enfermedades y por lo tanto es posible que, enfocándonos en el azul y el rojo de esta imagen, podamos determinar con más facilidad si la planta tiene _scab_.

### Canales con valores medios

Vistos ya algunos ejemplos de las plantas que podemos encontrar en nuestro conjunto de datos, hagamos lo mismo para las medias del conjunto de datos.

In [ ]:
# TODO: Discutir si es mejor mostrar más imágenes en vez de imagen + histograma
# en esta parte inicial.

# TODO: Sacar medias con np.mean y realizar los histogramas por cada canal
# y así concluir que forma tiene (usualmente normal).

# TODO: Ver métodos de detección de bordes y 